In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [8]:
train = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_case1.csv')

In [9]:
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_42,quality_0,quality_1,quality_5,quality_6,quality_8,quality_11,quality_12,period,multiple_model
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.644371,-0.023049,0.120567,-0.059816,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.362551,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.644371,-0.022932,0.023923,-0.059551,-0.011320,-0.044469,0.091401,-0.06688,0.272376,0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.080732,-0.023167,0.023923,-0.059949,0.031832,-0.044469,0.091401,-0.06688,0.442781,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.316156,-0.340504,-0.239957,-0.053688,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.482907,-0.023285,-0.072722,-0.059949,-0.060788,-0.044469,-0.043608,-0.06688,-2.965326,0
14996,-0.387084,-0.340504,-0.239957,-0.414143,-0.062749,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,-4.498974,0
14997,-0.078337,0.704060,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.764726,-0.023167,0.023923,-0.059757,-0.059735,-0.044469,0.091401,-0.06688,0.442781,0
14998,-0.333983,-0.156620,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,-1.090867,0


# not use quality

In [10]:
train_col = list(train.iloc[:, -9:-2].columns)

In [11]:
train.drop(columns=train_col, inplace=True)
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.442781,0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.442781,0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.272376,0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.442781,0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.442781,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.316156,-0.340504,-0.239957,-0.053688,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.491089,0.019092,0.482907,-2.965326,0
14996,-0.387084,-0.340504,-0.239957,-0.414143,-0.062749,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-4.498974,0
14997,-0.078337,0.704060,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.034927,-0.074159,0.628662,-0.114398,-0.080131,0.115262,0.019092,0.764726,0.442781,0
14998,-0.333983,-0.156620,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.045467,-0.074159,0.628662,-0.114398,-0.080131,-0.528213,-0.336841,-0.644371,-1.090867,0


In [12]:
train.to_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_case1.csv', index=False)